In [12]:
from scipy.fftpack import dst, idst
from IPython.display import Audio
import numpy as np

# Constants and Parameters
N = 150
L = 1.0
h = L / N
x = -L / 2 + h * np.arange(0, N) + h / 2  # Spatial positions
k = np.arange(1, N + 1) * np.pi / L

# Adjusting frequency for "middle C"
harmonics = [1, 9/8, 5/4, 3/2, 5/3]

# for h in harmonics:
f1 = 261.63*(harmonics[4])
lambda_ = 2 * np.pi * k * f1

dt = 2/np.max(lambda_)
N_ts_further_reduced = int(0.2*1/dt)
Samp_rate = 1/dt

# Define the initial position function (Initial pulse due to strike)
def H_0(x):
    strike_x = 0.1 * L
    return np.where(x < strike_x, x / strike_x, (L - x) / (L - strike_x))

# Initial conditions in spatial domain
A0 = H_0(x)
V0 = np.zeros_like(A0)

# Pickup location for sound recording
Pickup = N // 2

# Verlet Integration to solve for string displacement over time
S_verlet = np.zeros(N_ts_further_reduced)
An = dst(A0, type=1)
Vn = dst(V0, type=1)
A_prev = An
for n in range(N_ts_further_reduced):
    An_new = np.zeros_like(An)
    Vn_new = np.zeros_like(Vn)
    
    for k in range(1, N+1):
        lambda_k =  2 * np.pi**2 * k * f1
        A_n1 = 2 * An[k-1] - A_prev[k-1] - (lambda_k**2) * dt**2 * An[k-1]
        V_n1 = (A_n1 - A_prev[k-1]) / (2 * dt)
        
        An_new[k-1] = A_n1
        Vn_new[k-1] = V_n1

    A_prev = An.copy()
    An, Vn = An_new, Vn_new
    Hn = idst(An)
    S_verlet[n] = Hn[Pickup]

# Normalize the sound for playback
S_normalized_verlet = S_verlet / np.max(np.abs(S_verlet))

# Play the generated sound
Audio(S_normalized_verlet, rate=Samp_rate)


In [21]:
harmonics = [1, 9/8, 5/4, 3/2, 5/3]
def Waves(freq):
    # Constants and Parameters
    N = 150
    L = 1.0
    h = L / N
    x = -L / 2 + h * np.arange(0, N) + h / 2  # Spatial positions
    k = np.arange(1, N + 1) * np.pi / L

# Adjusting frequency for "middle C"
#harmonics = [1, 9/8, 5/4, 3/2, 5/3]

# for h in harmonics:
    f1 = 261.63*(freq)
    lambda_ = 2 * np.pi * k * f1

    dt = 2/np.max(lambda_)
    N_ts_further_reduced = int(0.2*1/dt)
    Samp_rate = 1/dt

    # Define the initial position function (Initial pulse due to strike)
    def H_0(x):
        strike_x = 0.1 * L
        return np.where(x < strike_x, x / strike_x, (L - x) / (L - strike_x))

    # Initial conditions in spatial domain
    A0 = H_0(x)
    V0 = np.zeros_like(A0)

    # Pickup location for sound recording
    Pickup = N // 2

    # Verlet Integration to solve for string displacement over time
    S_verlet = np.zeros(N_ts_further_reduced)
    An = dst(A0, type=1)
    Vn = dst(V0, type=1)
    A_prev = An
    for n in range(N_ts_further_reduced):
        An_new = np.zeros_like(An)
        Vn_new = np.zeros_like(Vn)
    
        for k in range(1, N+1):
            lambda_k =  2 * np.pi**2 * k * f1
            A_n1 = 2 * An[k-1] - A_prev[k-1] - (lambda_k**2) * dt**2 * An[k-1]
            V_n1 = (A_n1 - A_prev[k-1]) / (2 * dt)
        
            An_new[k-1] = A_n1
            Vn_new[k-1] = V_n1

        A_prev = An.copy()
        An, Vn = An_new, Vn_new
        Hn = idst(An)
        S_verlet[n] = Hn[Pickup]

# Normalize the sound for playback
    S_normalized_verlet = S_verlet / np.max(np.abs(S_verlet))
    
    Aud = Audio(S_normalized_verlet, rate=Samp_rate)

    return Aud

Waves(harmonics[4])

In [14]:
N = 150
L = 1.0
h = L / N
x = -L / 2 + h * np.arange(0, N) + h / 2  # Spatial positions
k = np.arange(1, N + 1) * np.pi / L

penta = []
for h in harmonics:
# Adjusting frequency for "middle C"
#harmonics = [1, 9/8, 5/4, 3/2, 5/3]

# for h in harmonics:
    f1 = 261.63*(h)
    lambda_ = 2 * np.pi * k * f1

    dt = 2/np.max(lambda_)
    N_ts_further_reduced = int(0.2*1/dt)
    Samp_rate = 1/dt

    # Define the initial position function (Initial pulse due to strike)
    def H_0(x):
        strike_x = 0.1 * L
        return np.where(x < strike_x, x / strike_x, (L - x) / (L - strike_x))

    # Initial conditions in spatial domain
    A0 = H_0(x)
    V0 = np.zeros_like(A0)

    # Pickup location for sound recording
    Pickup = N // 2

    # Verlet Integration to solve for string displacement over time
    S_verlet = np.zeros(N_ts_further_reduced)
    An = dst(A0, type=1)
    Vn = dst(V0, type=1)
    A_prev = An
    for n in range(N_ts_further_reduced):
        An_new = np.zeros_like(An)
        Vn_new = np.zeros_like(Vn)
    
        for k in range(1, N+1):
            lambda_k =  2 * np.pi**2 * k * f1
            A_n1 = 2 * An[k-1] - A_prev[k-1] - (lambda_k**2) * dt**2 * An[k-1]
            V_n1 = (A_n1 - A_prev[k-1]) / (2 * dt)
        
            An_new[k-1] = A_n1
            Vn_new[k-1] = V_n1

        A_prev = An.copy()
        An, Vn = An_new, Vn_new
        Hn = idst(An)
        S_verlet[n] = Hn[Pickup]

# Normalize the sound for playback
    S_normalized_verlet = S_verlet / np.max(np.abs(S_verlet))
    
    penta.append(Audio(S_normalized_verlet, rate=Samp_rate))

<ipython-input-14-30af196ff593>:44: RuntimeWarning: overflow encountered in double_scalars
  V_n1 = (A_n1 - A_prev[k-1]) / (2 * dt)
<ipython-input-14-30af196ff593>:43: RuntimeWarning: overflow encountered in double_scalars
  A_n1 = 2 * An[k-1] - A_prev[k-1] - (lambda_k**2) * dt**2 * An[k-1]
<ipython-input-14-30af196ff593>:43: RuntimeWarning: invalid value encountered in double_scalars
  A_n1 = 2 * An[k-1] - A_prev[k-1] - (lambda_k**2) * dt**2 * An[k-1]
